In [1]:
from delierium.Infinitesimals import prolongationODE
from delierium.JanetBasis import Janet_Basis

In [2]:
x = var('x')
y=function('y')

In [3]:
ode=4*diff(y(x),x,x)*y(x) - 3* (diff(y(x),x)**2)-12*(y(x)**3)

In [4]:
p=prolongationODE(ode, y, x)[0].expand()

In [5]:
for _ in p.operands():
    _.show()

-4*y(x)*D[0, 0](xi)(y(x), x)*diff(y(x), x)^3

4*y(x)*D[0, 0](phi)(y(x), x)*diff(y(x), x)^2

-8*y(x)*D[0, 1](xi)(y(x), x)*diff(y(x), x)^2

6*D[0](xi)(y(x), x)*diff(y(x), x)^3

-12*y(x)*D[0](xi)(y(x), x)*diff(y(x), x)*diff(y(x), x, x)

...

4*y(x)*D[0](phi)(y(x), x)*diff(y(x), x, x)

-8*y(x)*D[1](xi)(y(x), x)*diff(y(x), x, x)

4*y(x)*D[1, 1](phi)(y(x), x)

-6*D[1](phi)(y(x), x)*diff(y(x), x)

4*phi(y(x), x)*diff(y(x), x, x)

In [6]:
s1=solve(ode==0, diff(y(x),x,x))

In [7]:
s1=s1[0];s1.show();s1.lhs().show();s1.rhs().show()

diff(y(x), x, x) == 3/4*(4*y(x)^3 + diff(y(x), x)^2)/y(x)

diff(y(x), x, x)

3/4*(4*y(x)^3 + diff(y(x), x)^2)/y(x)

In [8]:
ode1=p.subs({s1.lhs() : s1.rhs()}).simplify()

In [9]:
ode1.expand().show()

-36*y(x)^3*D[0](xi)(y(x), x)*diff(y(x), x) - 4*y(x)*D[0, 0](xi)(y(x), x)*diff(y(x), x)^3 + 12*y(x)^3*D[0](phi)(y(x), x) - 24*y(x)^3*D[1](xi)(y(x), x) + 4*y(x)*D[0, 0](phi)(y(x), x)*diff(y(x), x)^2 - 8*y(x)*D[0, 1](xi)(y(x), x)*diff(y(x), x)^2 - 3*D[0](xi)(y(x), x)*diff(y(x), x)^3 - 24*phi(y(x), x)*y(x)^2 + 8*y(x)*D[0, 1](phi)(y(x), x)*diff(y(x), x) - 4*y(x)*D[1, 1](xi)(y(x), x)*diff(y(x), x) - 3*D[0](phi)(y(x), x)*diff(y(x), x)^2 + 4*y(x)*D[1, 1](phi)(y(x), x) - 6*D[1](phi)(y(x), x)*diff(y(x), x) + 3*phi(y(x), x)*diff(y(x), x)^2/y(x)

In [10]:
r1=ode1.expand().collect(diff(y(x),x)**3);r1.show()

-36*y(x)^3*D[0](xi)(y(x), x)*diff(y(x), x) + 12*y(x)^3*D[0](phi)(y(x), x) - 24*y(x)^3*D[1](xi)(y(x), x) + 4*y(x)*D[0, 0](phi)(y(x), x)*diff(y(x), x)^2 - 8*y(x)*D[0, 1](xi)(y(x), x)*diff(y(x), x)^2 - (4*y(x)*D[0, 0](xi)(y(x), x) + 3*D[0](xi)(y(x), x))*diff(y(x), x)^3 - 24*phi(y(x), x)*y(x)^2 + 8*y(x)*D[0, 1](phi)(y(x), x)*diff(y(x), x) - 4*y(x)*D[1, 1](xi)(y(x), x)*diff(y(x), x) - 3*D[0](phi)(y(x), x)*diff(y(x), x)^2 + 4*y(x)*D[1, 1](phi)(y(x), x) - 6*D[1](phi)(y(x), x)*diff(y(x), x) + 3*phi(y(x), x)*diff(y(x), x)^2/y(x)

In [11]:
r2=r1.collect(diff(y(x),x)**2);r2.show()

-36*y(x)^3*D[0](xi)(y(x), x)*diff(y(x), x) + 12*y(x)^3*D[0](phi)(y(x), x) - 24*y(x)^3*D[1](xi)(y(x), x) - (4*y(x)*D[0, 0](xi)(y(x), x) + 3*D[0](xi)(y(x), x))*diff(y(x), x)^3 - 24*phi(y(x), x)*y(x)^2 + 8*y(x)*D[0, 1](phi)(y(x), x)*diff(y(x), x) - 4*y(x)*D[1, 1](xi)(y(x), x)*diff(y(x), x) + (4*y(x)*D[0, 0](phi)(y(x), x) - 8*y(x)*D[0, 1](xi)(y(x), x) + 3*phi(y(x), x)/y(x) - 3*D[0](phi)(y(x), x))*diff(y(x), x)^2 + 4*y(x)*D[1, 1](phi)(y(x), x) - 6*D[1](phi)(y(x), x)*diff(y(x), x)

In [12]:
r3=r2.collect(diff(y(x),x)); r3.show()

12*y(x)^3*D[0](phi)(y(x), x) - 24*y(x)^3*D[1](xi)(y(x), x) - (4*y(x)*D[0, 0](xi)(y(x), x) + 3*D[0](xi)(y(x), x))*diff(y(x), x)^3 - 24*phi(y(x), x)*y(x)^2 + (4*y(x)*D[0, 0](phi)(y(x), x) - 8*y(x)*D[0, 1](xi)(y(x), x) + 3*phi(y(x), x)/y(x) - 3*D[0](phi)(y(x), x))*diff(y(x), x)^2 + 4*y(x)*D[1, 1](phi)(y(x), x) - 2*(18*y(x)^3*D[0](xi)(y(x), x) - 4*y(x)*D[0, 1](phi)(y(x), x) + 2*y(x)*D[1, 1](xi)(y(x), x) + 3*D[1](phi)(y(x), x))*diff(y(x), x)

In [13]:
r4=r3.collect(y(x)); r4.show()

-36*y(x)^3*D[0](xi)(y(x), x)*diff(y(x), x) - 4*y(x)*D[0, 0](xi)(y(x), x)*diff(y(x), x)^3 + 12*y(x)^3*D[0](phi)(y(x), x) - 24*y(x)^3*D[1](xi)(y(x), x) + 4*y(x)*D[0, 0](phi)(y(x), x)*diff(y(x), x)^2 - 8*y(x)*D[0, 1](xi)(y(x), x)*diff(y(x), x)^2 - 3*D[0](xi)(y(x), x)*diff(y(x), x)^3 - 24*phi(y(x), x)*y(x)^2 + 8*y(x)*D[0, 1](phi)(y(x), x)*diff(y(x), x) - 4*y(x)*D[1, 1](xi)(y(x), x)*diff(y(x), x) - 3*D[0](phi)(y(x), x)*diff(y(x), x)^2 + 4*y(x)*D[1, 1](phi)(y(x), x) - 6*D[1](phi)(y(x), x)*diff(y(x), x) + 3*phi(y(x), x)*diff(y(x), x)^2/y(x)

In [14]:
from IPython.core.debugger import set_trace
function('xi phi')
def infinitesimalsODE (ode, dependent, independent):
    prolongation = prolongationODE(ode, dependent, independent)[0].expand()
    s1  = solve(ode==0, diff(dependent(independent),independent,independent))
    ode1= prolongation.subs({s1[0].lhs() : s1[0].rhs()}).simplify()
    # so far it is checked manually and with mathematica
    l = [_ [0] for _ in ode1.coefficients(diff(dependent(independent), independent))]
    # remove hardcoded xi, phi
    janet = Janet_Basis(l, [xi, phi], [x, y])
    return janet

In [15]:
ode=4*diff(y(x),x,x)*y(x) - 3* diff(y(x),x)**2-12*y(x)**3
inf = infinitesimalsODE(ode, y ,x)